In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2


# if cannot import the modules, add the parent directory to system path might help

import os
import sys
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/'
sys.path.append(parent_dir)


import os
from utils.path import dir_HugeFiles
from utils.preprocessing import load
from utils.words import make_corpus_0, get_wordcount_list
from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv, print_time
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

#dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
#dic = load(dir_save)
random_seed = 2019

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [ ]:
dic = load(dir_save = '../big_data/dic_20190830.pickle')

ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))
desc = [i for i in ls if len(dic[i]['description'])<1]
print('drop %d recipes with no description' %(len(desc)))
print('now we are using recipe54k %d' % len(ls))

corpus_llist, corpus_list, corpus = make_corpus_0(dic, ['name','ingredients','directions'], ls)

In [3]:
X, knowns = get_wordcount_list(corpus_list, drop = False)

time: 1min 28s


In [212]:
def add_UNK(string, key):
    listt = string.split(' ')
    listt = [l if l in knowns else 'UNK' for l in listt]
    return listt
def rm_UNK(string, key):
    listt = string.split(' ')
    listt = [l for l in listt if l in knowns]
    return listt

for i, v in dic.items():
    if i in ls:
        for key in ['name','ingredients','directions']:
            dic[i][key+'_UNK'] = [rm_UNK(string, key) for string in v[key]]

time: 1min 20s


In [85]:
stats = []
# find world cuisine
world = set(['all world cuisine', 'world cuisine'])
# check the region tag
region = set(['latin american','european', 'asian', 'u . s .', 'african', 'australian and new zealander'])
for i, v in dic.items():
    if i in ls:
        # filter out the ' recipe'
        recipe = set([t[:-8] if t[-8:]==' recipes' else t for t in v['tags']])

        # if not world cusine
        if len( recipe & world ) == 0:
            regional_tag = 'unknown'
        else:
            check_region = recipe & region
            # if region not in the list
            if len(check_region) == 0:
                if 'italian' in recipe:
                    regional_tag = 'european'
                elif 'mexican' in recipe:
                    regional_tag = 'latin american'
                elif 'canadian' in recipe:
                    regional_tag = 'north american'
                elif 'middle eastern' in recipe:
                    regional_tag = 'asian'
                elif 'indian' in recipe:
                    regional_tag = 'asian'
                else:
                    regional_tag = 'unknown_region'
            # exact match
            elif len(check_region) == 1: 
                regional_tag = list(check_region)[0]
                if regional_tag == 'u . s .':
                    regional_tag = 'north american'
            # dual region
            else:
                regional_tag = 'dual_region'
        
        dic[i]['regional_tag'] = regional_tag
        stats.append(regional_tag)

time: 24 s


In [86]:
from collections import Counter
Counter(stats)

Counter({'unknown': 38098,
         'european': 5059,
         'latin american': 2590,
         'dual_region': 447,
         'north american': 3795,
         'asian': 3299,
         'australian and new zealander': 328,
         'african': 205,
         'unknown_region': 255})

time: 26.1 ms


In [ ]:
classes = {'european': 0, 'latin american': 1, 'north american': 2,'asian': 3,'australian and new zealander': 4, 'african': 5}
ls_region = [i for i in ls if not 'unknown' in dic[i]['regional_tag'] and not 'dual_region' in dic[i]['regional_tag']]
reviews = [v['name_UNK']+v['ingredients_UNK']+v['directions_UNK'] for i, v in dic.items() if i in ls_region]
labels = [classes[v['regional_tag']] for i, v in dic.items() if i in ls_region]

In [7]:
''' Because the performance of HAAT is similar to BoW, I will use BoW 

full_datasets = reviews, labels
save_pickle(dir_HugeFiles+'to_HATT/'+'regional_20190717.pickle', full_datasets, overwrite = True)
X_require_predict = [v['name_UNK']+v['ingredients_UNK']+v['directions_UNK'] for i, v in dic.items() if i in ls_region and i not in ls]
save_pickle(dir_HugeFiles+'to_HATT/'+'X_require_predict_20190717.pickle', X_require_predict, overwrite = True)
'''

" Because the performance of HAAT is similar to BoW, I will use BoW \nclasses = {'european': 0, 'latin american': 1, 'north american': 2,'asian': 3,'australian and new zealander': 4, 'african': 5}\nls_region = [i for i in ls if not 'unknown' in dic[i]['regional_tag'] and not 'dual_region' in dic[i]['regional_tag']]\nreviews = [v['name_UNK']+v['ingredients_UNK']+v['directions_UNK'] for i, v in dic.items() if i in ls_region]\nlabels = [classes[v['regional_tag']] for i, v in dic.items() if i in ls_region]\nfull_datasets = reviews, labels\nsave_pickle(dir_HugeFiles+'to_HATT/'+'regional_20190717.pickle', full_datasets, overwrite = True)\n"

time: 21.2 ms


In [244]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

time: 23.5 ms


### use the whole recipe

In [245]:
X, fn = get_wordcount_list([sum(recipe, []) for recipe in reviews])
y = np.array(labels)
config = {'fold':1}
ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 12)
fold = 0
for train_index, test_index in ss.split(X, y):
    X_test, y_test = X[test_index], y[test_index]
    X_train, y_train = X[train_index], y[train_index]
    if fold == config['fold']:
        break
    fold += 1
    
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size = 0.25)

best_f1 = 0
best_c = 1
for c in [0.1, 0.5, 1,3, 10, 100, 500]:
    model = ''
    model = LogisticRegression(C= c, multi_class='auto', solver = 'lbfgs')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    f1 =  round(classification_report(y_val, y_pred, output_dict=True)['weighted avg']['f1-score'] , 3)
    best_f1 = f1 if best_f1 < f1 else best_f1
    best_c = c if best_f1 < f1 else best_c
model = ''
model = LogisticRegression(C= best_c, multi_class='auto', solver = 'lbfgs')
model.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: l

              precision    recall  f1-score   support

           0       0.78      0.80      0.79      1012
           1       0.79      0.77      0.78       518
           2       0.66      0.70      0.68       759
           3       0.84      0.83      0.84       660
           4       0.24      0.12      0.16        66
           5       0.54      0.34      0.42        41

   micro avg       0.76      0.76      0.76      3056
   macro avg       0.64      0.59      0.61      3056
weighted avg       0.75      0.76      0.75      3056

time: 2min 2s


### use the recipe title

In [246]:
X, fn = get_wordcount_list([recipe[0] for recipe in reviews])
y = np.array(labels)

config = {'fold':1}
ss = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 12)
fold = 0
for train_index, test_index in ss.split(X, y):
    X_test, y_test = X[test_index], y[test_index]
    X_train, y_train = X[train_index], y[train_index]
    if fold == config['fold']:
        break
    fold += 1
    
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size = 0.25)

best_f1 = 0
best_c = 1
for c in [0.1, 0.5, 1, 3, 10, 100, 500]:
    model = ''
    model = LogisticRegression(C= c, multi_class='auto', solver = 'lbfgs')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    f1 =  round(classification_report(y_val, y_pred, output_dict=True)['weighted avg']['f1-score'] , 3)
    best_f1 = f1 if best_f1 < f1 else best_f1
    best_c = c if best_f1 < f1 else best_c
model = ''
model = LogisticRegression(C= best_c, multi_class='auto', solver = 'lbfgs')
model.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/lin

              precision    recall  f1-score   support

           0       0.76      0.86      0.81      1012
           1       0.87      0.79      0.83       518
           2       0.71      0.72      0.72       759
           3       0.84      0.80      0.82       660
           4       0.19      0.05      0.07        66
           5       0.92      0.59      0.72        41

   micro avg       0.78      0.78      0.78      3056
   macro avg       0.72      0.63      0.66      3056
weighted avg       0.77      0.78      0.77      3056

time: 30.8 s


/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [39]:
ls = range(20)

time: 18 ms


In [46]:
dic[2]['sections']

'Diet and Health->Healthy->Low-Carb Recipes;Dish Type->Main Dishes->Pork Main Dishes;World Cuisine->All World Cuisine->Italian Recipes;Dish Type->Main Dishes->Italian Main Dishes;Ingredient->Pork Recipes->Ground Pork;World Cuisine->Italian->Italian Main Dishes;Ingredient->Pork Recipes->Pork Shoulder;World Cuisine->Italian->Authentic Italian Recipes;Diet and Health->Low Carb Recipes->Low-Carb Main Dishes;Ingredient->Poultry->Sausage Recipes;Cooking Style->Gourmet->Gourmet Main Dishes;Ingredient->Poultry->Pork Recipes;Dish Type->Main Dishes->Gourmet Main Dishes;World Cuisine->All World Cuisine->European Recipes;Diet and Health->Weight-Loss Recipes->Low-Carb Recipes;Ingredient->Pork Recipes->Pork Main Dishes'

time: 26.6 ms


In [ ]:
'chinese', 'japanese', 'korean','indian','pakistani','bangladeshi','persian','filipino','indonesian','vietnamese','thai', 'korean', 'middle eastern','indian'
'italian', 'german', 'uk and ireland','french','mediterranean'

'canadian','indian','southern', 'u.s. recipes'
'mexican','south american','caribbean','latin american'

In [166]:
stats = []

# check the region tag
regional_tag2 = {'asian':set(['chinese', 'japanese', 'korean','indian','pakistani','bangladeshi',
                            'persian','filipino','indonesian','vietnamese','thai', 'korean', 
                            'middle eastern','indian']),
                 'european':set(['italian', 'german', 'uk and ireland','french']),
                 'north american':set(['canadian','southern','u . s .']),
                 'latin american': set(['mexican','south american','caribbean'])
              }

for i, v in dic.items():
    if i in ls_region:
        # filter out the ' recipe'
        recipe = set([t[:-8] if t[-8:]==' recipes' else t for t in v['tags']])
        if v['regional_tag'] in regional_tag2.keys():
            compare = regional_tag2[v['regional_tag']] & recipe
            if len(compare) == 1:
                dic[i]['regional_tag2'] = compare.pop()
            elif len(compare) > 1:
                dic[i]['regional_tag2'] = 'dual_region'
            else:
                dic[i]['regional_tag2'] = 'unknown'

time: 13.4 s


In [239]:
df = pd.DataFrame.from_dict(dic, orient = 'index')

time: 3.31 s


In [240]:
df.groupby(['regional_tag','regional_tag2'])['regional_tag2'].count()

regional_tag    regional_tag2 
asian           bangladeshi          3
                chinese            454
                dual_region         79
                filipino            95
                indian             637
                indonesian          21
                japanese           188
                korean             136
                middle eastern     281
                pakistani           17
                thai               203
                unknown           1157
                vietnamese          28
european        dual_region         10
                french             309
                german             117
                italian           2838
                uk and ireland     181
                unknown           1604
latin american  caribbean          146
                dual_region          2
                mexican           2024
                south american     107
                unknown            311
north american  canadian         

time: 39.3 ms


In [241]:
df = df[df['regional_tag']!='unknown']
df = df[df['regional_tag2']!='unknown']
df = df[df['regional_tag2']!='dual_region']
df = df[~df['regional_tag2'].isna()]

time: 108 ms


In [347]:
others = ['bangladeshi','filipino','indonesian','pakistani','vietnamese']
df['regional_tag2_'] = ['others' if v in others else v for v in df.regional_tag2]
df['regional_tag2_'] = ['north american' if v in ['canadian','u . s .'] else v for v in df.regional_tag2_]

time: 28.6 ms


In [348]:
gb = df.groupby(['regional_tag','regional_tag2_']).name_UNK.apply(list)
gb

regional_tag    regional_tag2_
asian           chinese           [[[chinese, chicken, salad, iii]], [[chinese, ...
                indian            [[[easy, peasy, pea, salad]], [[besan]], [[ind...
                japanese          [[[traditional, beef, sukiyaki]], [[japanese, ...
                korean            [[[quick, and, easy, kimchi, salad]], [[kimchi...
                middle eastern    [[[eggplants, in, red, sauce]], [[tahini, dres...
                others            [[[(, chicken, cooked, in, coconut, milk, )]],...
                thai              [[[thai, green, mango, salad]], [[shrimp, and,...
european        french            [[[french, apple, pie, with, cream, cheese, to...
                german            [[[german, rhubarb, streusel, cake]], [[kraut,...
                italian           [[[italian, sausage, tuscan, style]], [[easy, ...
                uk and ireland    [[[sweet, scottish, tablet]], [[scottish, minc...
latin american  caribbean         [[[cuban, r

time: 384 ms


In [350]:
datasets = {'asian':[], 'european':[], 'latin american':[],'north american':[]}
labels = {'asian':[], 'european':[], 'latin american':[],'north american':[]}
for (tag, tag2), row in gb.iteritems():
    datasets[tag] += row
    labels[tag] += [tag2]*len(row)

time: 26.5 ms


In [344]:
X, fn = get_wordcount_list([recipe[0] for recipe in datasets['north american']])
y = np.array(labels['north american'])

time: 173 ms


In [304]:
from sklearn.metrics import f1_score

time: 23.4 ms


In [345]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size = 0.25)

best_f1 = 0
best_c = 1
for c in [0.1, 0.5, 1, 3, 10, 100, 500]:
    model = ''
    model = LogisticRegression(C= c, multi_class='auto', solver = 'lbfgs')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    f1 =  f1_score(y_val, y_pred, average = 'micro')
    best_f1 = f1 if best_f1 < f1 else best_f1
    best_c = c if best_f1 < f1 else best_c
model = ''
model = LogisticRegression(C= best_c, multi_class='auto', solver = 'lbfgs')
model.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    canadian       0.69      0.31      0.43       154
     u . s .       0.80      0.95      0.87       442

   micro avg       0.79      0.79      0.79       596
   macro avg       0.74      0.63      0.65       596
weighted avg       0.77      0.79      0.75       596

time: 618 ms


/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


### everything classifier

In [351]:
datasets = []
labels = []
for (tag, tag2), row in gb.iteritems():
    datasets += row
    labels += [tag2]*len(row)

time: 30.4 ms


In [352]:
X, fn = get_wordcount_list([recipe[0] for recipe in datasets])
y = np.array(labels)

time: 1.97 s


In [354]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify = y_train, test_size = 0.25)

best_f1 = 0
best_c = 1
for c in [0.1, 0.5, 1, 3, 10, 100, 500]:
    model = ''
    model = LogisticRegression(C= c, multi_class='auto', solver = 'lbfgs', max_iter = 300)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    f1 =  f1_score(y_val, y_pred, average = 'weighted')
    best_f1 = f1 if best_f1 < f1 else best_f1
    best_c = c if best_f1 < f1 else best_c
model = ''
model = LogisticRegression(C= best_c, multi_class='auto', solver = 'lbfgs')
model.fit(np.concatenate([X_train, X_val]), np.concatenate([y_train, y_val]))
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


                precision    recall  f1-score   support

     caribbean       1.00      0.45      0.62        29
       chinese       0.82      0.67      0.74        91
        french       0.79      0.37      0.51        62
        german       1.00      0.39      0.56        23
        indian       0.82      0.70      0.76       128
       italian       0.86      0.85      0.85       568
      japanese       0.95      0.47      0.63        38
        korean       1.00      0.70      0.83        27
       mexican       0.89      0.83      0.86       405
middle eastern       0.85      0.41      0.55        56
north american       0.62      0.87      0.72       595
        others       0.94      0.52      0.67        33
south american       1.00      0.38      0.55        21
          thai       0.97      0.71      0.82        41
uk and ireland       0.75      0.42      0.54        36

     micro avg       0.77      0.77      0.77      2153
     macro avg       0.88      0.58      0.68 

/home/helena/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [341]:
from lightgbm import LGBMClassifier

time: 36.6 ms
